In [108]:
SMALL_SIZE, MEDIUM_SIZE = 9, 12
PROJECT_PATH = "/Users/donghur/Desktop/Research/Murthy/BM_GPU"

In [109]:
import sys
import numpy as np
from glob import glob
import random, matplotlib
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

sys.path.append(PROJECT_PATH)
from utils.data import Dataset


In [110]:
import utils.figure as ufigure
%load_ext autoreload
%autoreload 1
%aimport utils.data
%aimport utils.figure

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
Data = Dataset(PROJECT_PATH, 'final')
Data.load_data()

Loading INFO.yaml ...


  3%|▎         | 4/141 [00:00<00:03, 35.16it/s]

Finished loading INFO
Loading config.yaml ...
Finished loading config


 99%|█████████▉| 140/141 [00:10<00:00, 13.95it/s]


ValueError: cannot reshape array of size 1654768 into shape (12,10,15257)

In [106]:
# configuration
INFO = Data.info
INFO_values = Data.info_values
config = Data.config
skeleton = config['skeleton']
skeleton_color= config['skeleton_color']

# features
bp = Data.data_obj['bodypoints']
rot_bp = Data.data_obj['rotated_bodypoints']
angles = Data.data_obj['angles']
limbs = Data.data_obj['limbs']
angle_power = Data.data_obj['angle_power']
limb_power = Data.data_obj['limb_power']

# embeddings
all_embed = Data.data_obj['all_embeddings']
all_postural_embed = Data.data_obj['all_postural_embeddings']
all_kinematic_embed = Data.data_obj['all_kinematic_embeddings']

<hr><hr>

In [97]:
INFO_items = list(INFO.items())
INFO_items.sort(key=lambda x: x[1]['order'])
tot_limb= []

for key, file in tqdm(INFO_items):
    save_path = file['directory']
    bp = np.load(f"{PROJECT_PATH}/{save_path}/rotated_bodypoints.npy")
    bp = bp.astype("float64")
    num_fr, num_bp, _ = bp.shape
    likelihood = bp[:,:,2]
    
    ### Locate Bad Frames ###
    # check if below likelihood threshold
    (below_thresh_fr, below_thresh_marker) = np.where(likelihood < config['likelihood_thresh'])
    cnt = collections.Counter(below_thresh_fr)
    cnt_array = np.array(list(cnt.items()))
    # check if above marker threshold
    try:
        # set low likelihood fr as bad fr
        bad_fr_idx = np.where(cnt_array[:,1] > config['marker_thresh'])[0]
        bad_fr = cnt_array[bad_fr_idx,0]
        # set nan fr as bad fr
        nan_fr, _, _ = np.where( np.isnan(bp) )
        unique_nan_fr = np.unique(nan_fr)
        bad_fr = np.concatenate([bad_fr, unique_nan_fr])
        bad_fr = np.unique(bad_fr)
        # append pads
        padded_fr = np.array([ list(range(fr-config['bad_fr_pad'], fr+config['bad_fr_pad']+1)) for fr in bad_fr])
        disregard_fr = np.unique(padded_fr.flatten())
        disregard_fr = disregard_fr[(disregard_fr >= 0) & (disregard_fr < num_fr)]
        good_fr_idx = np.array([True]*num_fr)
        good_fr_idx[disregard_fr] = False
        good_fr = np.where(good_fr_idx==True)[0]
    except:
        bad_fr = np.array([])
        disregard_fr = np.array([])
        good_fr = np.arange(num_fr)

    # TODO: modify proportion of good fr
    file['good_fr'] = good_fr.tolist()
    file['bad_fr'] = bad_fr.tolist()
    file['disregard_fr'] = disregard_fr.tolist()
    
    
    # ********* fix ***********
    limbs = np.zeros((num_fr, len(config['limbs'])))
    for i, limb_pts in enumerate(config['limbs']):
        bp_good_fr = bp[good_fr,:,:]

        limb_i = bp_good_fr[:,limb_pts,:]
        
        limbs[good_fr,i] = np.sqrt((limb_i[:,0,0]-limb_i[:,1,0])**2 + (limb_i[:,0,1]-limb_i[:,1,1])**2)
    inf_fr,_ = np.where(np.isinf(limbs))
    if len(inf_fr) != 0:
        break

        
#     tot_limb.append(limbs[good_fr,:])

In [107]:
num_fr,_,_ = rot_bp.shape

nan_fr,_,_ = np.where(np.isnan(rot_bp))
nan_fr = np.unique(nan_fr)
good_fr = np.array([True]*num_fr)
good_fr[nan_fr] = False

np.max(bp[good_fr,:,:])

4022.0